In [1]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils import *
from pkg.planning.filtering.lattice_model.data_utils import *

DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"
GSCENE_FOLDER = "gscene"

VISUALIZE = False

In [2]:
ROBOT_TYPE_NAME="indy7gripper"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
ROBOT_DATA_ROOT_FAILMORE = ROBOT_DATA_ROOT+"-failmore"
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)

In [3]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN_FILTERED = dataset_list[:10]
DATASET_TEST_FILTERED = dataset_list[10:15]
print(DATASET_TRAIN_FILTERED)
print(DATASET_TEST_FILTERED)

dataset_list_fails = sorted([folder for folder 
                             in os.listdir(ROBOT_DATA_ROOT_FAILMORE) 
                             if not folder.startswith(".")])
DATASET_TRAIN_FAILMORE = dataset_list_fails[:10]
DATASET_TEST_FAILMORE = dataset_list_fails[10:15]
print(DATASET_TRAIN_FAILMORE)
print(DATASET_TEST_FAILMORE)

['20210920-194228', '20210920-224003', '20210921-014626', '20210921-044409', '20210921-074756', '20210921-103631', '20210921-134210', '20210921-165251', '20210921-200258', '20210921-230351']
['20210922-015645', '20210922-050510', '20210922-080941', '20210922-112600', '20210922-142917']
['20210922-213904', '20210922-220459', '20210922-222948', '20210922-225336', '20210922-231228', '20210922-233339', '20210922-235716', '20210923-002032', '20210923-004132', '20210923-010440']
['20210923-012622', '20210923-014705', '20210923-020906', '20210923-023448', '20210923-030221']


In [4]:
DATA_ROOT = ROBOT_DATA_ROOT
DATASET = DATASET_TRAIN_FILTERED

def get_data_pairs(data_root, datasets):
    data_pairs = []
    for dataset in datasets:
        file_list = sorted(os.listdir(os.path.join(data_root, dataset, GRASP_FOLDER)))
        for file in file_list:
            data_pairs.append(
                (os.path.join(data_root, dataset, GRASP_FOLDER, file), 
                 os.path.join(data_root, dataset, ARM10_FOLDER, file), 
                 os.path.join(data_root, dataset, ARM05_FOLDER, file), 
                 os.path.join(data_root, dataset, FULLS_FOLDER, file), 
                 os.path.join(data_root, dataset, GSCENE_FOLDER, file)))
    return data_pairs

In [5]:
data_pairs_train_filtered = get_data_pairs(ROBOT_DATA_ROOT, DATASET_TRAIN_FILTERED)
print("train set: {}".format(len(data_pairs_train_filtered)))

data_pairs_test_filtered = get_data_pairs(ROBOT_DATA_ROOT, DATASET_TEST_FILTERED)
print("test set: {}".format(len(data_pairs_test_filtered)))

data_pairs_train_failmore = get_data_pairs(ROBOT_DATA_ROOT_FAILMORE, DATASET_TRAIN_FAILMORE)
print("train fail set: {}".format(len(data_pairs_train_failmore)))

data_pairs_test_failmore = get_data_pairs(ROBOT_DATA_ROOT_FAILMORE, DATASET_TEST_FAILMORE)
print("test fail set: {}".format(len(data_pairs_test_failmore)))

train set: 10002
test set: 5005
train fail set: 10035
test fail set: 5016


In [6]:
from pkg.planning.filtering.lattice_model.latticizer_py import *
from pkg.planning.constraint.constraint_common import *
import itertools

ltc_full = Latticizer_py(WDH=(3, 3, 3), L_CELL=0.05, OFFSET_ZERO=(1.5, 1.5, 1.5))
ltc_effector = Latticizer_py(WDH=(1, 1, 1), L_CELL=0.05, OFFSET_ZERO=(0.5, 0.5, 0.5))
ltc_arm_05 = Latticizer_py(WDH=(2, 2, 2), L_CELL=0.05, OFFSET_ZERO=(0.5, 1.0, 1.0))
ltc_arm_10 = Latticizer_py(WDH=(2, 2, 2), L_CELL=0.10, OFFSET_ZERO=(0.5, 1.0, 1.0))


In [7]:
from pkg.utils.test_scripts import *

ROBOT_TYPE = {e.name: e for e in RobotType}[ROBOT_TYPE_NAME]
ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene
crob.home_pose = HOME_POSE
crob.home_dict = list2dict(HOME_POSE, gscene.joint_names)
HOME_DICT = crob.home_dict
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)


from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


In [ ]:
for data_pairs in [data_pairs_train_filtered, data_pairs_test_filtered, data_pairs_train_failmore, data_pairs_test_failmore]:
    for data_pair in data_pairs:
        grasp_file, arm10_file, arm5_file, fscene_file, gscene_file =data_pair
        print("======== {} ========".format(gscene_file))
        grasp_dict_load = load_pickle(grasp_file)
        arm10_dict_load = load_pickle(arm10_file)
        arm05_dict_load = load_pickle(arm5_file)
        fscene_dict_load = load_pickle(fscene_file)
        scene_dat = load_pickle(gscene_file)
        T_loal = scene_dat["T_loal"]
        pscene.clear_subjects()
        gscene.clear_non_fixed()
        gscene.clear_link("base_link")
        initial_state = load_saved_scene(pscene, gscene_file, VISUALIZE=VISUALIZE, 
                                         GRIP_DEPTH=GRIP_DEPTH, CLEARANCE = 1e-3)

        success_reach, success_retrieve = grasp_dict_load["reach"], grasp_dict_load["retrieve"]
        time_reach, success_retrieve = grasp_dict_load["reach_time"], grasp_dict_load["retrieve_time"]

        grasp_dict = {}
        arm_05_dict = {}
        arm_10_dict = {}
        full_scene_dict = {}
        ltc_effector.clear()
        ltc_arm_05.clear()
        ltc_arm_10.clear()
        ltc_full.clear()

        ##################################################
        ################ CHECK REGENERATION ################

        actor = pscene.actor_dict['grip0']
        actor.redundancy['w'] = (-np.deg2rad(45), np.deg2rad(45))# default redundancy changed
        T_ba = np.matmul(T_loal, actor.Toff_lh)
        obj, handle = find_match(pscene, actor, T_ba, crob.home_dict)

        actor_vertinfo_list, object_vertinfo_list, actor_Tinv_dict, object_Tinv_dict = \
                                            gcheck.get_grasping_vert_infos(actor, obj, T_loal, HOME_DICT)

        if all([ROBOT_NAME in lname for lname in actor_Tinv_dict.keys()]):
            tool_Tinv_dict = actor_Tinv_dict
            tool_vertinfo_list = actor_vertinfo_list
            target_vertinfo_list = object_vertinfo_list
            T_end_effector = T_loal
        elif all([ROBOT_NAME in lname for lname in object_Tinv_dict.keys()]):
            tool_Tinv_dict = object_Tinv_dict
            tool_vertinfo_list = object_vertinfo_list
            target_vertinfo_list = actor_vertinfo_list
            T_end_effector = SE3_inv(T_loal)
        else:
            raise("Invaild TOOL_LINK_BUNDLE")
        T_end_joint = T_end_effector

        r, th, h = cart2cyl(*T_end_effector[:3,3])
        Tref = SE3(Rot_axis(3, th), T_end_effector[:3,3])
        Tref_base = SE3(Tref[:3,:3], (0, 0, shoulder_height))
        
#         obj_names = [obj.geometry.name] + obj.geometry.children
#         target_names= [item[0] for item in target_vertinfo_list if item[0] not in obj_names]
#         tool_names = [item[0] for item in tool_vertinfo_list]

#         ltc_effector.convert_vertices(tool_vertinfo_list, Tref=Tref)
#         ltc_effector.convert_vertices(target_vertinfo_list, Tref=Tref)
#         ltc_arm_05.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
#         ltc_arm_10.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
#         ltc_full.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
#         ltc_full.convert_vertices(actor_vertinfo_list, Tref=Tref_base)

#         grasp_dict["tar"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in target_names if tname in ltc_effector.coll_idx_dict])))
#         grasp_dict["tool"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in tool_names if tname in ltc_effector.coll_idx_dict])))
#         grasp_dict["obj"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in obj_names if tname in ltc_effector.coll_idx_dict])))
#         grasp_dict["T_end_effector"], grasp_dict["T_end_joint"], grasp_dict["Tref_base"]  = T_end_effector, T_end_joint, Tref_base
#         grasp_dict["reach"], grasp_dict["retrieve"] = success_reach, success_retrieve
#         grasp_dict["reach_time"], grasp_dict["retrieve_time"] = time_reach, success_retrieve

#         arm_05_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_05.coll_idx_dict])))
#         arm_05_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_05.coll_idx_dict])))
#         arm_05_dict["T_end_effector"], arm_05_dict["T_end_joint"], arm_05_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
#         arm_05_dict["reach"], arm_05_dict["retrieve"] = success_reach, success_retrieve

#         arm_10_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_10.coll_idx_dict])))
#         arm_10_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_10.coll_idx_dict])))
#         arm_10_dict["T_end_effector"], arm_10_dict["T_end_joint"], arm_10_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
#         arm_10_dict["reach"], arm_10_dict["retrieve"] = success_reach, success_retrieve

#         full_scene_dict["tar"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in target_names if tname in ltc_full.coll_idx_dict])))
#         full_scene_dict["tool"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in tool_names if tname in ltc_full.coll_idx_dict])))
#         full_scene_dict["obj"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in obj_names if tname in ltc_full.coll_idx_dict])))
#         full_scene_dict["T_end_effector"], full_scene_dict["T_end_joint"], full_scene_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
#         full_scene_dict["reach"], full_scene_dict["retrieve"] = success_reach, success_retrieve

#         assert compare_dict(grasp_dict_load,grasp_dict)
#         assert compare_dict(arm05_dict_load,arm_05_dict)
#         assert compare_dict(arm10_dict_load,arm_10_dict)
#         assert compare_dict(fscene_dict_load,full_scene_dict)

        ################ CHECK REGENERATION ################
        ##################################################

        ##################################################
        ################## MAKE NEW DATA ##################

        (object_geo_list, object_T2end_dict), (actor_geo_list, actor_T2end_dict) = \
                        gcheck.get_geolist_tflist_pairs(actor, obj, HOME_DICT)

        (object_geo_list, object_T2end_dict), (actor_geo_list, actor_T2end_dict) = \
                        gcheck.get_geolist_tflist_pairs(actor, obj, HOME_DICT)


        obj_names = obj.geometry.get_family()

        group_name = ROBOT_NAME
        T_br = gscene.get_tf(crob.get_robot_base_dict()[group_name], Q=crob.home_dict,
                             from_link=obj.geometry.link_name)
        T_re = np.matmul(SE3_inv(T_br), T_loal)
        T_tool_to_rob = T_re
        tool_geo_list, tool_T2end_dict = actor_geo_list, actor_T2end_dict
        T_tar_to_rob = SE3_inv(T_br)
        target_geo_list, target_T2end_dict = object_geo_list, object_T2end_dict

        grasp_dict = {}
        arm_05_dict = {}
        arm_10_dict = {}
        full_scene_dict = {}
        ltc_effector.clear()
        ltc_arm_05.clear()
        ltc_arm_10.clear()
        ltc_full.clear()

        r, th, h = cart2cyl(*T_tool_to_rob[:3, 3])
        T_rl = SE3(Rot_axis(3, th), T_re[:3, 3])  # in robot base link coordinate
        target_names = [gtem.name for gtem in target_geo_list if gtem.name not in obj_names]
        tool_names = [gtem.name for gtem in tool_geo_list]

        ## Convert effector
        T_gl_list = []
        gtem_list = target_geo_list + tool_geo_list
        for gtem in gtem_list:
            if gtem.link_name in tool_T2end_dict:
                T_rg = matmul_series(T_tool_to_rob, tool_T2end_dict[gtem.link_name], gtem.Toff)
            else:
                T_rg = matmul_series(T_tar_to_rob, target_T2end_dict[gtem.link_name], gtem.Toff)
            T_lg = np.matmul(SE3_inv(T_rl), T_rg)
            T_gl = SE3_inv(T_lg)
            T_gl_list.append(T_gl)

        ltc_effector.convert_vertices_approx(gtem_list, T_gl_list)

        ## Convert env
        link_env = [lname for lname in gscene.link_names
                    if lname not in pscene.robot_chain_dict[group_name]["link_names"]]
        gtems_env = [gtem for gtem in gscene
                     if gtem.collision and gtem.link_name in link_env]
        Trl_base = SE3(T_rl[:3, :3], (0, 0, shoulder_height))  # in robot base link coordinate
        T_bl = np.matmul(T_br, Trl_base)
        T_lb = SE3_inv(T_bl)
        Tlist_env = {lname: gscene.get_tf(lname, crob.home_dict) for lname in link_env}
        T_gl_list_env = []
        for gtem in gtems_env:
            T_lg = matmul_series(T_lb, Tlist_env[gtem.link_name], gtem.Toff)
            T_gl = SE3_inv(T_lg)
            T_gl_list_env.append(T_gl)
        ltc_arm_05.convert_vertices_approx(gtems_env, T_gl_list_env)
        ltc_arm_10.convert_vertices_approx(gtems_env, T_gl_list_env)

        T_gl_list_tool = []
        for gtem in tool_geo_list:
            T_rg = matmul_series(T_tool_to_rob, tool_T2end_dict[gtem.link_name], gtem.Toff)
            T_lg = matmul_series(T_lb, T_br, T_rg)
            T_gl = SE3_inv(T_lg)
            T_gl_list_tool.append(T_gl)
        ltc_full.convert_vertices_approx(gtems_env, T_gl_list_env)
        ltc_full.convert_vertices_approx(tool_geo_list, T_gl_list_tool)

        grasp_dict["tar"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in target_names if tname in ltc_effector.coll_idx_dict])))
        grasp_dict["tool"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in tool_names if tname in ltc_effector.coll_idx_dict])))
        grasp_dict["obj"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in obj_names if tname in ltc_effector.coll_idx_dict])))
        grasp_dict["T_end_effector"], grasp_dict["T_end_joint"], grasp_dict["Tref_base"]  = T_loal, T_end_joint, Tref_base
        grasp_dict["reach"], grasp_dict["retrieve"] = success_reach, success_retrieve
        grasp_dict["reach_time"], grasp_dict["retrieve_time"] = time_reach, success_retrieve

        arm_05_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_05.coll_idx_dict])))
        arm_05_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_05.coll_idx_dict])))
        arm_05_dict["T_end_effector"], arm_05_dict["T_end_joint"], arm_05_dict["Tref_base"] = T_loal, T_end_joint, Tref_base
        arm_05_dict["reach"], arm_05_dict["retrieve"] = success_reach, success_retrieve

        arm_10_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_10.coll_idx_dict])))
        arm_10_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_10.coll_idx_dict])))
        arm_10_dict["T_end_effector"], arm_10_dict["T_end_joint"], arm_10_dict["Tref_base"] = T_loal, T_end_joint, Tref_base
        arm_10_dict["reach"], arm_10_dict["retrieve"] = success_reach, success_retrieve

        full_scene_dict["tar"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in target_names if tname in ltc_full.coll_idx_dict])))
        full_scene_dict["tool"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in tool_names if tname in ltc_full.coll_idx_dict])))
        full_scene_dict["obj"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in obj_names if tname in ltc_full.coll_idx_dict])))
        full_scene_dict["T_end_effector"], full_scene_dict["T_end_joint"], full_scene_dict["Tref_base"] = T_loal, T_end_joint, Tref_base
        full_scene_dict["reach"], full_scene_dict["retrieve"] = success_reach, success_retrieve

        save_pickle(grasp_file, grasp_dict)
        save_pickle(arm5_file, arm_05_dict)
        save_pickle(arm10_file, arm_10_dict)
        save_pickle(fscene_file, full_scene_dict)
        ################## MAKE NEW DATA ##################
        ##################################################

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000000.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000001.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000002.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000003.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000004.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000005.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000006.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000007.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000073.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000074.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000075.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000076.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000077.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000078.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000079.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000080.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000146.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000147.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000148.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000149.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000150.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000151.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000152.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000153.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000219.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000220.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000221.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000222.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000223.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000224.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000225.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000226.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000292.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000293.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000294.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000295.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000296.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000297.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000298.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000299.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-194228/gscene/000

...

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000385.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000386.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000387.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000388.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000389.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000390.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000391.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000392.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000458.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000459.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000460.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000461.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000462.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000463.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000464.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000465.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000531.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000532.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000533.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000534.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000535.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000536.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000537.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000538.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000604.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000605.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000606.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000607.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000608.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000609.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000610.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000611.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000

======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000677.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000678.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000679.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000680.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000681.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000682.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000683.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000684.pkl ========
======== /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper/20210920-224003/gscene/000

### debugging

In [ ]:
find_match(pscene, actor, T_ba, crob.home_dict)

In [ ]:
gscene.clear_highlight()

In [ ]:
gscene.add_highlight_axis("actor", "point", link_name="base_link", center=T_ba[:3,3], orientation_mat=T_ba[:3,:3])

In [ ]:
obj_0 = pscene.subject_dict['obj_0']
handle = obj_0.action_points_dict['obj_0_hdl_tp_a']
pscene.add_handle_axis("handle", handle)

In [ ]:
actor.check_type(handle)
binder_redundancy = actor.get_redundancy()
binder_T = T_ba
handle_T = handle.get_tf_handle(crob.home_dict)
handle_redundancy = handle.get_redundancy()
margin_mat = get_binding_margins(handle_T, binder_T, handle_redundancy, binder_redundancy)

In [ ]:
np.min(margin_mat)

In [9]:
### Comparing errors

In [152]:
def compare_idc(name, idc_strict, idc_approx, raise_error=False, allow_ratio=0.1):
    diff_inv = list(set(idc_approx) - set(idc_strict))
    if len(diff_inv) > 0: 
        msg = name + " cell over {} / {}".format(len(diff_inv), len(idc_strict))
        if raise_error:
            raise(RuntimeError(msg))
        else:
            print(msg)
            return list(diff_inv)
    if len(idc_strict)==0:
        return
    diff = tuple(np.transpose(sorted(set(idc_strict) - set(idc_approx))))

    if float(len(diff))/len(idc_strict) > allow_ratio: 
        msg =name + " diff too much {}".format(round(
            float(len(diff))/len(idc_strict), 3))
        if raise_error:
            raise(RuntimeError(msg))
        else:
            print(msg)      
            return list(diff  )

In [154]:
compare_idc("grasp_tar", grasp_dict_load['tar'], grasp_dict['tar'])
compare_idc("grasp_tool", grasp_dict_load['tool'], grasp_dict['tool'])
compare_idc("grasp_obj", grasp_dict_load['obj'], grasp_dict['obj'])

diff_idc = compare_idc("arm05_tar", arm05_dict_load['tar'], arm_05_dict['tar'], allow_ratio=0.0)
compare_idc("arm05_tool", arm05_dict_load['tool'], arm_05_dict['tool'])

diff_idc = compare_idc("arm10_tar", arm10_dict_load['tar'], arm_10_dict['tar'], allow_ratio=0.0)
compare_idc("arm05_tool", arm10_dict_load['tool'], arm_10_dict['tool'])

diff_idc = compare_idc("fscene_tar", fscene_dict_load['tar'], full_scene_dict['tar'], allow_ratio=0.0)
diff_idc = compare_idc("fscene_tool", fscene_dict_load['tool'], full_scene_dict['tool'])
compare_idc("fscene_obj", fscene_dict_load['obj'], full_scene_dict['obj'])

grasp_tar diff too much 0.101
grasp_tool diff too much 0.295
grasp_obj diff too much 0.25
arm05_tar diff too much 0.05
arm10_tar diff too much 0.063
fscene_tar diff too much 0.04
fscene_tool diff too much 0.268
fscene_obj diff too much 0.4


[160285, 160286, 160287, 163765, 163766, 163767]

In [25]:
T_loal

array([[ 4.6029374e-01, -8.2606637e-01,  3.2518303e-01,  2.7106124e-01],
       [-6.7468327e-01, -5.6357282e-01, -4.7664252e-01,  6.7925942e-01],
       [ 5.7700270e-01,  1.1124338e-16, -8.1674224e-01,  2.0268646e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)

In [21]:
scene_dat = load_pickle(gscene_file)
T_loal = scene_dat["T_loal"]
        
diff_idc = grasp_dict_load['tar']
r, th, h = cart2cyl(*T_loal[:3, 3])
T_rl = SE3(Rot_axis(3, th), T_re[:3, 3])  # in robot base link coordinate

In [22]:
ltc_centers = ltc_effector.get_centers().reshape((-1,3))
diff_centers = np.matmul(Trl_base[:3,:3], ltc_centers[diff_idc].transpose()).transpose() + Trl_base[:3,3]

In [23]:
gscene.clear_highlight()

In [24]:
for i_c, center in enumerate(diff_centers):
    gscene.add_highlight_axis("diff", "{:05}".format(i_c), link_name=crob.get_robot_base_dict()[ROBOT_NAME], 
                              center=center, orientation_mat=np.identity(3), axis=None)